In [21]:
%pip install langchain-core langchain-community faiss-cpu tiktoken PyPDF opendatasets sentence-transformers --quiet
%pip install python-dotenv langchain-together --quiet


[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [22]:
import dotenv

dotenv.load_dotenv()

True

In [23]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import ChatPromptTemplate
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
import warnings
from langchain_together import TogetherEmbeddings
from langchain_community.llms import Together

# warnings.filterwarnings("ignore")

In [24]:
pdf_loader = PyPDFLoader('./content/1706.03762.pdf')
pages = pdf_loader.load_and_split()

In [25]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1024,
    chunk_overlap  = 128,
    length_function = len,
)
chunks = text_splitter.split_documents(pages)

In [43]:
chunks[14]

Document(metadata={'source': './content/1706.03762.pdf', 'page': 3}, page_content='extremely small gradients4. To counteract this effect, we scale the dot products by1√dk.\n3.2.2 Multi-Head Attention\nInstead of performing a single attention function with dmodel-dimensional keys, values and queries,\nwe found it beneficial to linearly project the queries, keys and values htimes with different, learned\nlinear projections to dk,dkanddvdimensions, respectively. On each of these projected versions of\nqueries, keys and values we then perform the attention function in parallel, yielding dv-dimensional\n4To illustrate why the dot products get large, assume that the components of qandkare independent random\nvariables with mean 0and variance 1. Then their dot product, q·k=Pdk\ni=1qiki, has mean 0and variance dk.\n4')

In [27]:
Embeddings = TogetherEmbeddings(
    model="togethercomputer/m2-bert-80M-8k-retrieval")

vectorStore = FAISS.from_texts([str(chunk) for chunk in chunks], Embeddings)

In [28]:
## Without LLM
question = """
what is the difference between supervised and unsupervised learning?
"""
docs = vectorStore.similarity_search(question, k = 2)

docs

[Document(page_content="page_content='(section 5.4), learning rates and beam size on the Section 22 development set, all other parameters\nremained unchanged from the English-to-German base translation model. During inference, we\n9' metadata={'source': './content/1706.03762.pdf', 'page': 8}"),
 Document(page_content="page_content='trained the base models for a total of 100,000 steps or 12 hours. For our big models,(described on the\nbottom line of table 3), step time was 1.0 seconds. The big models were trained for 300,000 steps\n(3.5 days).\n5.3 Optimizer\nWe used the Adam optimizer [ 20] with β1= 0.9,β2= 0.98andϵ= 10−9. We varied the learning\nrate over the course of training, according to the formula:\nlrate =d−0.5\nmodel·min(step_num−0.5, step _num·warmup _steps−1.5) (3)\nThis corresponds to increasing the learning rate linearly for the first warmup _steps training steps,\nand decreasing it thereafter proportionally to the inverse square root of the step number. We used\nwarmup _s

In [29]:
prompt = """
    Using this information:
    \n
    Context: {context}
    \n
    Answer the following:
    \n
    Question: {question}
    \n
    Answer:\n
    """
prompt = ChatPromptTemplate.from_template(prompt)

In [30]:
retriever = vectorStore.as_retriever(kwargs={"k": 3})

llm = Together(model="meta-llama/Meta-Llama-3-70B-Instruct-Turbo")


In [44]:
qa_chain = RetrievalQA.from_chain_type(llm=llm,
                                       retriever=retriever,
                                       return_source_documents=True,
                                       chain_type_kwargs={
                                           'prompt': prompt, 
                                           'verbose': True
                                           },
                                       verbose=True
                                       )

In [45]:
question = "what is the transformer model architecture? "

result = qa_chain({"query": question})



> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Human: 
    Using this information:
    

    Context: page_content='Figure 1: The Transformer - model architecture.
The Transformer follows this overall architecture using stacked self-attention and point-wise, fully
connected layers for both the encoder and decoder, shown in the left and right halves of Figure 1,
respectively.
3.1 Encoder and Decoder Stacks
Encoder: The encoder is composed of a stack of N= 6 identical layers. Each layer has two
sub-layers. The first is a multi-head self-attention mechanism, and the second is a simple, position-
wise fully connected feed-forward network. We employ a residual connection [ 11] around each of
the two sub-layers, followed by layer normalization [ 1]. That is, the output of each sub-layer is
LayerNorm( x+ Sublayer( x)), where Sublayer( x)is the function implemented by the sub-layer
itself. To facil

In [33]:
result["query"]

'what is the transformer model architecture? '

In [34]:
print(result["result"])

 The Transformer model architecture consists of an encoder and a decoder, both composed of a stack of N=6 identical layers. Each layer has two sub-layers: a multi-head self-attention mechanism and a simple, position-wise fully connected feed-forward network. The output of each sub-layer is LayerNorm(x + Sublayer(x)), where Sublayer(x) is the function implemented by the sub-layer itself. The model uses residual connections around each of the two sub-layers, followed by layer normalization. The encoder and decoder stacks are shown in the left and right halves of Figure 1, respectively.


In [35]:
print(result["source_documents"][0].page_content)

page_content='Figure 1: The Transformer - model architecture.
The Transformer follows this overall architecture using stacked self-attention and point-wise, fully
connected layers for both the encoder and decoder, shown in the left and right halves of Figure 1,
respectively.
3.1 Encoder and Decoder Stacks
Encoder: The encoder is composed of a stack of N= 6 identical layers. Each layer has two
sub-layers. The first is a multi-head self-attention mechanism, and the second is a simple, position-
wise fully connected feed-forward network. We employ a residual connection [ 11] around each of
the two sub-layers, followed by layer normalization [ 1]. That is, the output of each sub-layer is
LayerNorm( x+ Sublayer( x)), where Sublayer( x)is the function implemented by the sub-layer
itself. To facilitate these residual connections, all sub-layers in the model, as well as the embedding
layers, produce outputs of dimension dmodel = 512 .
Decoder: The decoder is also composed of a stack of N= 6ide

In [36]:
print("Your Question: ", result["query"])
print()
print("Your answer: ", result["result"])
print()
print("Sources: ", result["source_documents"])

Your Question:  what is the transformer model architecture? 

Your answer:   The Transformer model architecture consists of an encoder and a decoder, both composed of a stack of N=6 identical layers. Each layer has two sub-layers: a multi-head self-attention mechanism and a simple, position-wise fully connected feed-forward network. The output of each sub-layer is LayerNorm(x + Sublayer(x)), where Sublayer(x) is the function implemented by the sub-layer itself. The model uses residual connections around each of the two sub-layers, followed by layer normalization. The encoder and decoder stacks are shown in the left and right halves of Figure 1, respectively.

Sources:  [Document(page_content="page_content='Figure 1: The Transformer - model architecture.\nThe Transformer follows this overall architecture using stacked self-attention and point-wise, fully\nconnected layers for both the encoder and decoder, shown in the left and right halves of Figure 1,\nrespectively.\n3.1 Encoder and De

In [37]:
docs[0].page_content

"page_content='(section 5.4), learning rates and beam size on the Section 22 development set, all other parameters\nremained unchanged from the English-to-German base translation model. During inference, we\n9' metadata={'source': './content/1706.03762.pdf', 'page': 8}"